In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('sql-example').getOrCreate()
sc = spark.sparkContext

21/12/01 22:41:22 WARN Utils: Your hostname, jesus-Aspire-A514-52 resolves to a loopback address: 127.0.1.1; using 192.168.1.54 instead (on interface wlp2s0)
21/12/01 22:41:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/01 22:41:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Se leen como Dataframe, por simplicidad
weather = spark.read.csv("nycweather.csv", mode="DROPMALFORMED",inferSchema=True)
print(weather.columns)
# Las columnas se nombran automáticamente 
# Los campos del datset son la fecha, la temperatura media en grados
# Celsius y la precipitación (litros?)
# Cambiamos los nombres por claridad
weather = weather.withColumnRenamed('_c0', 'Datetime').withColumnRenamed('_c1','Temp').withColumnRenamed('_c2','Precip')
print(weather.columns)
# Schema inferido
weather.printSchema()
# Veamos los datos
weather.show()

['_c0', '_c1', '_c2']
['Datetime', 'Temp', 'Precip']
root
 |-- Datetime: string (nullable = true)
 |-- Temp: integer (nullable = true)
 |-- Precip: double (nullable = true)

+----------+----+------+
|  Datetime|Temp|Precip|
+----------+----+------+
|2013-01-01|   1|   0.0|
|2013-01-02|  -2|   0.0|
|2013-01-03|  -2|   0.0|
|2013-01-04|   1|   0.0|
|2013-01-05|   3|   0.0|
|2013-01-06|   4|   0.0|
|2013-01-07|   5|   0.0|
|2013-01-08|   6|   0.0|
|2013-01-09|   7|   0.0|
|2013-01-10|   7|   0.0|
|2013-01-11|   6| 13.97|
|2013-01-12|   7|  0.51|
|2013-01-13|   8|   0.0|
|2013-01-14|   8|  2.29|
|2013-01-15|   3|  3.05|
|2013-01-16|   2| 17.53|
|2013-01-17|   4|   0.0|
|2013-01-18|  -1|   0.0|
|2013-01-19|   5|   0.0|
|2013-01-20|   6|   0.0|
+----------+----+------+
only showing top 20 rows



In [4]:
# Solo nos interesa la parte del año (date) y tener las tres partes por separado (Year, Month, Day)
from pyspark.sql.functions import to_date, split
date_col = to_date(weather['Datetime'])
splited_col = split(date_col,"-")
year_col = splited_col.getItem(0)
month_col = splited_col.getItem(1)
day_col = splited_col.getItem(2)
weather = weather.withColumn('Year', year_col).withColumn('Month', month_col).withColumn('Day', day_col)
# Delete Datetime column
weather = weather.drop('Datetime')
# Veamos los campos nuevos
weather.show()
# Schema inferido
weather.printSchema()

+----+------+----+-----+---+
|Temp|Precip|Year|Month|Day|
+----+------+----+-----+---+
|   1|   0.0|2013|   01| 01|
|  -2|   0.0|2013|   01| 02|
|  -2|   0.0|2013|   01| 03|
|   1|   0.0|2013|   01| 04|
|   3|   0.0|2013|   01| 05|
|   4|   0.0|2013|   01| 06|
|   5|   0.0|2013|   01| 07|
|   6|   0.0|2013|   01| 08|
|   7|   0.0|2013|   01| 09|
|   7|   0.0|2013|   01| 10|
|   6| 13.97|2013|   01| 11|
|   7|  0.51|2013|   01| 12|
|   8|   0.0|2013|   01| 13|
|   8|  2.29|2013|   01| 14|
|   3|  3.05|2013|   01| 15|
|   2| 17.53|2013|   01| 16|
|   4|   0.0|2013|   01| 17|
|  -1|   0.0|2013|   01| 18|
|   5|   0.0|2013|   01| 19|
|   6|   0.0|2013|   01| 20|
+----+------+----+-----+---+
only showing top 20 rows

root
 |-- Temp: integer (nullable = true)
 |-- Precip: double (nullable = true)
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Day: string (nullable = true)



In [5]:
# Registramos la tabla
weather.registerTempTable("weather")

/home/jesus/universidad/Infraestructuras_Computacionales/practicas/TP2/sparkenv/lib/python3.8/site-packages/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


In [7]:
weather.createOrReplaceTempView("weather")

Obtenemos los días en los que llovió, ordenados por la temperatura (de mas a menos temperatura)

In [8]:
hottest_with_precip = spark.sql("SELECT * FROM weather WHERE Precip > 0.0 ORDER BY Temp DESC")
hottest_with_precip.show()

+----+------+----+-----+---+
|Temp|Precip|Year|Month|Day|
+----+------+----+-----+---+
|  27|  7.87|2013|   07| 23|
|  27|  5.59|2013|   07| 08|
|  27|  1.27|2013|   08| 09|
|  27|  5.84|2013|   07| 09|
|  27|   6.1|2013|   06| 27|
|  27|  1.27|2013|   06| 26|
|  27|  1.52|2013|   07| 22|
|  26| 21.59|2013|   06| 02|
|  26|  0.25|2013|   09| 10|
|  26|  1.27|2013|   09| 02|
|  26|  0.25|2013|   08| 27|
|  26| 10.92|2013|   08| 28|
|  26| 40.64|2013|   09| 12|
|  26| 13.46|2013|   07| 03|
|  25|  0.25|2013|   06| 17|
|  25|  2.03|2013|   07| 02|
|  25|  0.25|2013|   07| 29|
|  24| 21.34|2013|   07| 01|
|  24| 11.68|2013|   08| 08|
|  24|  1.27|2013|   08| 12|
+----+------+----+-----+---+
only showing top 20 rows



Calculamos el total de precipitaciones por mes

In [9]:
group_by_month = spark.sql("SELECT ROUND(SUM(Precip)) AS PrepTotal, Month  FROM weather GROUP BY Month ORDER BY PrepTotal DESC")
group_by_month.show()

+---------+-----+
|PrepTotal|Month|
+---------+-----+
|    257.0|   06|
|    203.0|   05|
|    108.0|   02|
|     80.0|   11|
|     80.0|   12|
|     75.0|   09|
|     73.0|   03|
|     72.0|   07|
|     72.0|   08|
|     70.0|   01|
|     33.0|   04|
|      9.0|   10|
+---------+-----+

